In [1]:
from helper import *

In [2]:
categories = {'pets' : 'familie/haustiere', 'politics' : 'politik',
              'economy' : 'wirtschaft', 'travel' : 'konsum/reisen',
              'health' : 'gesundheit/medizin-krankheit', 'food' : 'ernahrung/lebensmittel',
              'parenting' : 'familie/erziehung', 'realestate' : 'wohnen/eigentum',
              'career' : 'arbeit', 'home' : 'wohnen/bauen-renovieren',
              'nature' : 'umwelt/umweltpolitik'}
website = 'https://www.beobachter.ch'

In [3]:
for category, path in categories.items():
    print(f'Working with {category} category')

    lines = []
    page_num = 1

    while True:
        url = f'{website}/{path}?page={page_num}#page'
        page = requests.get(url)    
        soup = BeautifulSoup(page.content, 'html.parser')
        
        if page_num == 1:
            try:
                limit = int(soup.find('div', class_='_2fQ8qDhH _1deHe_oF').
                            find_all('a', class_='page-loader-btn ShimA1UP')[-1].text)
            except AttributeError:
                limit = 1
            print(f'Number of pages = {limit}')
            
        divs = soup.find_all('div', class_='_23Qt4XY-')
        if divs == []:
            break

        for div in divs[1:]:
            sub_divs = div.find_all('a', class_='_2p02b0mw teaser-m-default')

            for sub_div in sub_divs:
                teaser_headline = sub_div.find('div', class_='_2QZpcT0M _3du5gVzi _3iAkatGy')
                teaser = sub_div.find('div', class_='_2QcjsKyb M6MeFidn')
                
                if None in (teaser_headline, teaser):
                    continue
                
                # going to the page of the article to get keywords
                href = sub_div['href']
                try:
                    article = requests.get(website + href)
                    article_soup = BeautifulSoup(article.content, 'html.parser')
                    keywords = ' '.join(article_soup.find('meta', attrs={'name':'news_keywords'})['content'].split(','))
                except TypeError:
                    keywords = ''

                teaser_headline = teaser_headline.text.strip()
                teaser = teaser.text.strip()[:-5] #[:-5] to throw away the word "Mehr"
                
                lines.append(teaser_headline)
                lines.append(teaser)
                lines.append(keywords)

        print(f'Scraped {len(lines)} lines')

        page_num += 1
        if page_num > limit:
            break
            
    print(f'Finished working with {category} category. Scraped {len(lines)} lines\n')
    write_to_file(lines, category, 'beobachter_ch')

Working with pets category
Number of pages = 10
Scraped 36 lines
Scraped 72 lines
Scraped 105 lines
Scraped 141 lines
Scraped 177 lines
Scraped 213 lines
Scraped 249 lines
Scraped 285 lines
Scraped 321 lines
Scraped 324 lines
Finished working with pets category. Scraped 324 lines

324 lines of pets category have been scraped from beobachter_ch

______________________________________________________________________________


Working with politics category
Number of pages = 17
Scraped 24 lines
Scraped 60 lines
Scraped 96 lines
Scraped 132 lines
Scraped 168 lines
Scraped 204 lines
Scraped 240 lines
Scraped 276 lines
Scraped 312 lines
Scraped 348 lines
Scraped 384 lines
Scraped 420 lines
Scraped 456 lines
Scraped 486 lines
Scraped 504 lines
Scraped 540 lines
Scraped 552 lines
Finished working with politics category. Scraped 552 lines

552 lines of politics category have been scraped from beobachter_ch

______________________________________________________________________________


Working